In [ ]:
import ee
import geemap.core as geemap
ee.Authenticate()
ee.Initialize(project='ee-guchvygucy')

In [ ]:
# Function to apply scale factors
def applyScaleFactors(image):
    opticalBands = (image.select('SR_B.')
                    .multiply(0.0000275)
                    .add(-0.2))
    thermalBands = (image.select('ST_B.*')
                    .multiply(0.00341802)
                    .add(149.0))
    return (image.addBands(opticalBands, None, True)
            .addBands(thermalBands, None, True))
# Define the AOI (Area of Interest) using a FeatureCollection
aoi = ee.FeatureCollection("projects/ee-guchvygucy/assets/Kurnool_test")

# Load the Landsat 8 image collection
l8 = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
      .filterDate('2019-01-01', '2019-01-31')
      .filterBounds(aoi)
      .map(applyScaleFactors)
      .sort('CLOUD_COVER', True)
      .median())



# Visualization parameters
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

m = geemap.Map()
m.add_layer(l8.clip(aoi), visualization, 'True Color (432)')
m


# NDVI calculation
ndvi = l8.normalizedDifference(['SR_B5', 'SR_B4'])

# Visualize NDVI
ndviPalette = ['blue', 'white', 'green']
m.addLayer(ndvi.clip(aoi), {'min': -1, 'max': 1, 'palette': ndviPalette}, 'NDVI')

# EVI calculation
evi = l8.expression(
    '2.5 * ((NIR - Red) / (NIR + 6 * Red - 7.5 * Blue + 1))', {
        'NIR': l8.select('SR_B5'),
        'Red': l8.select('SR_B4'),
        'Blue': l8.select('SR_B2')
    }
)

# Visualize EVI
eviPalette = ['blue', 'white', 'green']
m.addLayer(evi.clip(aoi), {'min': -1, 'max': 1, 'palette': eviPalette}, 'EVI')
m

savi = l8.expression(
    '((NIR - Red) / (NIR + Red + 0.5))', {
        'NIR': l8.select('SR_B5'),  # Near-Infrared band
        'Red': l8.select('SR_B4')   # Red band
    }
)

# Visualize SAVI
saviPalette = ['blue', 'white', 'green']
m.addLayer(savi.clip(aoi), {'min': -1, 'max': 1, 'palette': saviPalette}, 'SAVI')

# MSAVI calculation
msavi = l8.expression(
    '((2 * NIR + 1 - sqrt((2 * NIR + 1)**2 - 8 * (NIR - Red))) / 2)', {
        'NIR': l8.select('SR_B5'),  # Near-Infrared band
        'Red': l8.select('SR_B4')   # Red band
    }
)

# Visualize MSAVI
msaviPalette = ['blue', 'white', 'green']
m.addLayer(msavi.clip(aoi), {'min': -1, 'max': 1, 'palette': msaviPalette}, 'MSAVI')

# Load a digital elevation model (DEM) dataset
dem = ee.Image("USGS/SRTMGL1_003")

# Compute slope (in degrees)
slope = ee.Terrain.slope(dem)

# Compute aspect (in degrees clockwise from north)
aspect = ee.Terrain.aspect(dem)

# Visualize slope (in degrees)
m.addLayer(slope.clip(aoi), {'min': 0, 'max': 45, 'palette': ['blue', 'yellow', 'orange', 'red']}, 'Slope (degrees)')

# Visualize aspect (in degrees)
m.addLayer(aspect.clip(aoi), {'min': 0, 'max': 360, 'palette': ['blue', 'yellow', 'orange', 'red']}, 'Aspect (degrees)')

# Calculate area
area = (slope.divide(57.2958)
        .tan()
        .add(0.01)
        .pow(-1)
        .multiply(0.0001)
        .resample('bilinear'))

# Calculate the Topographic Index (TI)
twi = area.divide(slope.tan()).log()

# Visualize TWI
twiPalette = ['blue', 'white', 'green']
m.addLayer(twi.clip(aoi), {'min': -1, 'max': 1, 'palette': twiPalette}, 'TWI')

# NDMI calculation
ndmi = l8.normalizedDifference(['SR_B5', 'SR_B6'])

# Visualize NDMI
ndmiPalette = ['blue', 'white', 'green']
m.addLayer(ndmi.clip(aoi), {'min': -1, 'max': 1, 'palette': ndmiPalette}, 'NDMI')

# MNDWI calculation
mndwi = l8.normalizedDifference(['SR_B3', 'SR_B6'])

# Visualize MNDWI
mndwiPalette = ['blue', 'white', 'green']
m.addLayer(mndwi.clip(aoi), {'min': -1, 'max': 1, 'palette': mndwiPalette}, 'MNDWI')

# MSI calculation
msi = l8.expression(
    '((SWIR) / (NIR))', {
        'SWIR': l8.select('SR_B6'),
        'NIR': l8.select('SR_B5')
    }
)

# Visualize MSI
msiPalette = ['blue', 'white', 'green']
m.addLayer(msi.clip(aoi), {'min': -1, 'max': 1, 'palette': msiPalette}, 'MSI')

# Selecting bands for visualization
l8_nir = l8.select('SR_B5')  # Near-Infrared band (B4)
l8_swir = l8.select('SR_B6')  # Shortwave Infrared band (B5)
l8_tir = l8.select('ST_B10')  # Thermal Infrared band (B6)

# Adding layers for visualization
m.addLayer(l8_nir.clip(aoi), {'min': 0, 'max': 0.3, 'palette': ['black', 'white']}, 'NIR')
m.addLayer(l8_swir.clip(aoi), {'min': 0, 'max': 0.3, 'palette': ['black', 'white']}, 'SWIR')
m.addLayer(l8_tir.clip(aoi), {'min': 0, 'max': 300, 'palette': ['black', 'white']}, 'TIR')

m

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [ ]:
# Function to apply scale factors
def applyScaleFactors(image):
    opticalBands = (image.select('SR_B.')
                    .multiply(0.0000275)
                    .add(-0.2))
    thermalBands = (image.select('ST_B.*')
                    .multiply(0.00341802)
                    .add(149.0))
    return (image.addBands(opticalBands, None, True)
            .addBands(thermalBands, None, True))

# Function to mask clouds and cloud shadows
def cloudMask(image):
    # Define cloud shadow and cloud bitmasks (Bits 3 and 5)
    cloudShadowBitmask = (1 << 3)
    cloudBitmask = (1 << 5)

    # Select the Quality Assessment (QA) band for pixel quality information
    qa = image.select('QA_PIXEL')

    # Create a binary mask to identify clear conditions (both cloud and cloud shadow bits set to 0)
    mask = qa.bitwiseAnd(cloudShadowBitmask).eq(0).And(qa.bitwiseAnd(cloudBitmask).eq(0))

    # Update the original image, masking out cloud and cloud shadow-affected pixels
    return image.updateMask(mask)

# Load the Landsat 8 image collection
l8 = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
      .filterDate('2019-03-01', '2019-03-31')
      .filterBounds(aoi)
      .map(applyScaleFactors)
      .map(cloudMask)
      .sort('CLOUD_COVER', True)
      .median()
      .clip(aoi))

# Visualization parameters
visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

# Add true color layer to the map
m.addLayer(l8.clip(aoi), visualization, 'True Color (432)')

# Calculate Normalized Difference Vegetation Index (NDVI)
ndvi = l8.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# Define NDVI Visualization Parameters
ndviPalette = {
    'min': -1,
    'max': 1,
    'palette': ['blue', 'white', 'green']
}

# Add NDVI layer to the map
m.addLayer(ndvi, ndviPalette, 'NDVI')

# Calculate minimum and maximum NDVI values within the AOI
ndviMin = ndvi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).values().get(0)

ndviMax = ndvi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).values().get(0)

print("Minimum NDVI:", ndviMin.getInfo())
print("Maximum NDVI:", ndviMax.getInfo())

# Calculate Fractional Vegetation (FV) and Emissivity (EM)
fv = ndvi.expression('((ndvi-ndmin)/(ndmax-ndmin))**2',{
    'ndvi': ndvi,'ndmax': ndviMax.getInfo(),'ndmin':ndviMin.getInfo()
}).rename('FV')
em = fv.multiply(0.004).add(0.986).rename('EM')

# Calculate Land Surface Temperature (LST)
thermal = l8.select('ST_B10').rename('thermal')
lst = thermal.expression(
    '(TB / (1 + (0.00115 * (TB / 1.438)) * log(em))) - 273.15', {
        'TB': thermal.select('thermal'),
        'em': em
    }
).rename('LST')

# Calculate minimum and maximum LST values within the AOI
lstMin = lst.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).values().get(0)

lstMax = lst.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).values().get(0)

print("Minimum LST:", lstMin.getInfo())
print("Maximum LST:", lstMax.getInfo())
# Calculate Soil Moisture Index (SMI)
smi = lst.expression(
    '((Tmax - lst) / (Tmax - Tmin))', {
        'Tmax': lstMax.getInfo(),
        'Tmin': lstMin.getInfo(),
        'lst': lst
    }
).rename('SMI')

# Calculate minimum and maximum SMI values within the AOI
smiMin = smi.reduceRegion(
    reducer=ee.Reducer.min(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).values().get(0)

smiMax = smi.reduceRegion(
    reducer=ee.Reducer.max(),
    geometry=aoi,
    scale=30,
    maxPixels=1e9
).values().get(0)

print("Minimum SMI:", smiMin.getInfo())
print("Maximum SMI:", smiMax.getInfo())

# Add Land Surface Temperature (LST) layer to the map
r = geemap.Map()
r.addLayer(smi , {'min': smiMin.getInfo(), 'max': smiMax.getInfo(), 'palette': ['blue', 'white', 'green']}, 'SMI')
r

Minimum NDVI: -0.3246334751063296
Maximum NDVI: 0.8110604954156091
Minimum LST: 30.748568928130112
Maximum LST: 62.83266070564531
Minimum SMI: 0
Maximum SMI: 1


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [ ]:
task = ee.batch.Export.image.toDrive(image=smi,
                                     description='SMI_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=aspect,
                                     description='ASPECT_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=dem,
                                     description='DEM_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=evi,
                                     description='EVI_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=mndwi,
                                     description='MNDWI_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=msavi,
                                     description='MSAVI_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=msi,
                                     description='MSI_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=ndmi,
                                     description='NDMI_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=ndvi,
                                     description='NDVI_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=savi,
                                     description='SAVI_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=l8_nir,
                                     description='NIR_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=slope,
                                     description='SLOPE_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()

In [ ]:
task = ee.batch.Export.image.toDrive(image=l8_swir,
                                     description='SWIR_export',
                                     folder='GEE_exports_kurnool',  # Change this to your desired folder
                                     scale=30,
                                     region=aoi.geometry(),
                                     maxPixels=1e9)
task.start()